# Trigger ArcticDEM Download 

In [ ]:
import ee
import os
import geopandas as gpd
import glob
import eeconvert
%config Completer.use_jedi = False

In [ ]:
ee.Initialize()

### Settings

#### Paths

In [ ]:
f = r'' # Add Path to your Polygon file here

### Function 

In [ ]:
def get_ArcticDEM_slope():
    dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")
    slope = ee.Terrain.slope(dem)
    return slope

In [ ]:
def get_ArcticDEM_rel_el(kernel_size=100, offset=50, factor=300):
    dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")
    conv = dem.convolve(ee.Kernel.circle(kernel_size, 'meters'))
    diff = (dem.subtract(conv).add(ee.Image.constant(offset)).multiply(ee.Image.constant(factor)).toInt16())
    return diff

In [ ]:
def download_ee(filepath, ee_layer, GDrive_dir='AI-CORE/slope', scale=2, buffersize=5000, suffix='_slope'):
    
    name = os.path.basename(filepath).split('.')[0] + suffix
    gdf = gpd.read_file(filepath).to_crs(crs='EPSG:4326')
    
    ee_fc = eeconvert.gdfToFc(gdf)
    buffered_geom = ee_fc.geometry().buffer(5000)
    
    dl = ee.batch.Export.image.toDrive(
        image=ee_layer,
        description=name,
        folder=GDrive_dir,
        fileNamePrefix=name,
        dimensions=None,
        region=buffered_geom,
        scale=2,
        crs=None,
        crsTransform=None,
        maxPixels=1e13,
        shardSize=None,
        fileDimensions=None,
        skipEmptyTiles=None,
        fileFormat=None,
        formatOptions=None)
    
    dl.start()
    
    return dl

#### Access GEE data layer

In [ ]:
slope = get_ArcticDEM_slope()

In [ ]:
rel_el_100 = get_ArcticDEM_rel_el(kernel_size=100)

### Loop over datasets
#### Slope

#### Run full loop over slope 

In [ ]:
task = download_ee(f, slope)

In [ ]:
print('Running Tasks:')
if task.status()['state'] == 'RUNNING':
    task.status()['description']

In [ ]:
print('Completed Tasks:')
if task.status()['state'] == 'COMPLETED':
    task.status()['description']

#### Elevation

#### Run full loop over elevation

In [ ]:
task_el = download_ee(f, rel_el_100, GDrive_dir='AI-CORE/elevation', suffix='_rel_el_100_scaled')

In [ ]:
print('Running Tasks:')
if task_el.status()['state'] == 'RUNNING':
    task_el.status()['description']

In [ ]:
print('Completed Tasks:')
if task_el.status()['state'] == 'COMPLETED':
    task_el.status()['description']